In [1]:
import imageio
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from natsort import natsorted
from ipywidgets import interact
from scipy.special import spherical_jn as sph_jn, spherical_yn as sph_yn, lpn
from mpl_toolkits.axisartist.axislines import SubplotZero

# Set the default font size and family
plt.rc('font', family='sans-serif', size=14)

# Number of gif frames
nframes = 80

In [2]:
# Constants
hbar = 197.3269631
m = 938.272013

# Radius Grid
rmax, nsteps = 20, 2000
step = rmax/nsteps
r = np.linspace(0, rmax, nsteps+2)

# Potential Parameters
R0 = 3.0
R1 = 8.0
V0 = 50.0
V1 = 10.0

# Potential function
def V(r):
    if( r < R0 ): return -V0
    elif( r < R1 ): return V1
    else: return 0

In [3]:
# Numerov Integrator
def numerov(q, u, n, x, h, E, L):
    nodes, c = 0, h*h/12.
    f0, f1 = 0., q(E,L,x+h)
    for i in range(n):
        x += h
        f2 = q(E,L,x+h)
        u.append((2*(1-5*c*f1)*u[i+1] - (1+c*f0)*u[i])/(1+c*f2))
        f0, f1 = f1, f2
        if (u[-1]*u[-2] < 0.0): nodes += 1
    return u, nodes

# Schrodinger Equation in Numerov form
def q(E, L, r):
    return (2 * m / (hbar**2)*(E - V(r)) - L*(L+1)/(r*r))

# Wave Function and its Derivative at xm
def wf(M, xm, E, L, h):
    c = (h*h)/6.
    wfup, nup = numerov(q, [0,.1], M, 0., h, E, L)
    dup = ((1+c*q(E,L,xm+h))*wfup[-1] - (1+c*q(E,L,xm-h))*wfup[-3])/(h+h)
    return wfup, dup/wfup[-2]

# Computation
def compute( E, Lmax ):
    k, ps = np.sqrt(2 * E * m) / hbar , np.zeros(Lmax+1) 

    Lrange = range(Lmax + 1)
    jl, dj = sph_jn(Lrange, k*rmax, False), sph_jn(Lrange, k*rmax, True) # (j_l, j_l')
    nl, dn = sph_yn(Lrange, k*rmax, False), sph_yn(Lrange, k*rmax, True) # (n_l, n_l')

    uL = []
    for L in range(Lmax+1):
        u, g = wf(nsteps, rmax, E, L, step)
        uL.append(u)
        x = np.arctan(((g*rmax-1)*jl[L] - k*rmax*dj[L])/((g*rmax-1)*nl[L] - k*rmax*dn[L]))
        ps[L] = x

    theta, sigma = np.linspace(0., np.pi, 100), []
    cos, La = np.cos(theta), np.arange(1,2*Lmax+2,2)
    for x in cos:
        pl = lpn(Lmax, x)[0]
        fl = La * np.exp(1j*ps) * np.sin(ps) * pl / k
        sigma.extend(np.real(fl)**2 + np.imag(fl)**2)

    # Get the index at which r = r0 and at which r = r1
    index_r0, index_r1 = int(R0/step), int(R1/step)

    # Get the wavefunction max value in the internal and external region
    A, B = max(uL[0][:index_r0]), max(uL[0][index_r1:])

    # Calculate the transmission coefficient
    T = (A/B)**2

    return np.asarray( uL ), np.asarray( ps ), np.asarray( sigma ), T

In [4]:
def plot( e0=5, v0=50, v1=10, r0=3, r1=8 ):
    global V0, V1, R0, R1
    V0, V1, R0, R1 = v0, v1, r0, r1
    
    Emin, Emax, n = 0.5 * e0, 1.5 * e0, 500
    x, y_p, y_s, y_t = np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n)
    for idx in range( n ):
        e = Emin + idx*(Emax - Emin)/n
        _, ps, cross, T = compute(e, 0)
        x[idx], y_p[idx], y_s[idx], y_t[idx] =  e, ps[0], np.sum(cross[0]), T

    # Check in y array if the value jumps by more than 1, if so remove it
    for i in range( 1, len(y_p) ):
        if( abs(y_p[i] - y_p[i-1]) > 1 ):
            y_p[i:] = [x-(y_p[i] - y_p[i-1]) for x in y_p[i:]]

    # Get the wavefunction
    uL, _, _, _ = compute(e0, 1)
    pot = [V(x) for x in r]

    # Find the closest index to e0
    idx = np.abs(x - e0).argmin()
    ps, cross, T = y_p[idx], y_s[idx], y_t[idx]

    # Create 4 different figures
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Scattering Properties')

    # Plot the wavefunction
    axs[0, 0].plot(r, 10*uL[0]/max(abs(uL[0])),label="L = 0")
    axs[0, 0].plot(r, 10*uL[1]/max(abs(uL[1])),label="L = 1")
    axs[0, 0].plot(r,pot)
    axs[0, 0].set_xlim(0,15)
    axs[0, 0].set_xlabel('r (fm)')
    axs[0, 0].set_ylabel('V (MeV)')

    # Plot the transmission coefficient
    axs[0, 1].plot(x, y_t)
    axs[0, 1].scatter([e0], [T], color='red')
    axs[0, 1].set_xlim(Emin, Emax)
    axs[0, 1].set_xlabel('E (MeV)')
    axs[0, 1].set_yscale('log')
    axs[0, 1].set_ylabel('Transmission Coefficient')

    # Plot the phase shift
    axs[1, 0].plot(x, y_p)
    axs[1, 0].scatter([e0], [ps], color='red')
    axs[1, 0].set_xlim(Emin, Emax)
    axs[1, 0].set_xlabel('E (MeV)')
    axs[1, 0].set_ylabel('Phase Shift')

    # Plot the cross section
    axs[1, 1].plot(x, y_s)
    axs[1, 1].scatter([e0], [cross], color='red')
    axs[1, 1].set_xlim(Emin, Emax)
    #axs[1, 1].set_yscale('log')
    axs[1, 1].set_xlabel('E (MeV)')
    axs[1, 1].set_ylabel('Cross Section (mb)')

    plt.show()

In [5]:
interact( plot, e0=(1,800), v0=(0,100), v1=(0,100), r0=(0,20), r1=(0,20) )

interactive(children=(IntSlider(value=5, description='e0', max=800, min=1), IntSlider(value=50, description='v…

<function __main__.plot(e0=5, v0=50, v1=10, r0=3, r1=8)>

In [6]:
for idx in range( nframes ):

    e0_min = 2.
    e0_max = 10.

    # Select the potential and the radius
    V0, V1 = 50, 10
    R0, R1 = 3, 8
    e0 = e0_min + idx * (e0_max - e0_min) / nframes

    # Plot the solution for different values of E, Vo and a
    energies = np.linspace(1, 800, 400)
    shift = np.zeros( energies.shape )
    transmission = np.zeros( energies.shape )

    Emin, Emax, n = e0_min, e0_max, 500
    energies, shift, transmission = np.zeros(n), np.zeros(n), np.zeros(n)
    for j in range( n ):
        e = Emin + j * (Emax - Emin) / n
        _, ps, _, T = compute(e, 0)
        energies[j], shift[j], transmission[j] =  e, ps[0], T

    # Check in y array if the value jumps by more than 1, if so remove it
    for i in range( 1, len(shift) ):
        if( abs(shift[i] - shift[i-1]) > 1 ):
            shift[i:] = [x-(shift[i] - shift[i-1]) for x in shift[i:]]

    uL, _, _, _ = compute(e0, 0)
    pot = [V(x) for x in r]

    # Find the closest index to e0
    index = np.abs(energies - e0).argmin()
    ps, T = shift[index], transmission[index]

    # Divide the wavefunction in u1, u2, u3 basing on r0 and r1
    index_r0 = np.argmin( np.abs(r - R0) )
    index_r1 = np.argmin( np.abs(r - R1) )

    u_1 = uL[0][:index_r0]
    u_2 = uL[0][index_r0:index_r1]
    u_3 = uL[0][index_r1:]

    # Create two figures
    fig, axs = plt.subplots( 2, 1, figsize=(7, 9) )
    ax1, ax2 = axs[0], axs[0].twinx( )

    # Plot the transmission probability
    ax1.plot(energies, transmission, color='tab:blue')
    ax1.scatter(e0, T, color='tab:blue')
    ax1.set_yscale( "log" )
    ax1.set_xlabel('Center-of-Mass Energy (MeV)')
    ax1.set_ylabel(r'$|\frac{A}{C}| ^{2}$', color='tab:blue', rotation=0, labelpad=20, fontsize=15)

    # Plot the phase shift
    ax2.plot(energies, shift, color='tab:red')
    ax2.scatter(e0, ps, color='tab:red')
    ax2.set_ylabel(r'$\delta _0$', color='tab:red', rotation=0, labelpad=20)

    ax1.set_title(r'V$_0$ = -50 MeV, r$_0$ = 3 fm, V$_1$ = 10 MeV, r$_1$ = 8 fm')
    ax1.set_xlim(e0_min, e0_max)

    axs[1].plot(r[:index_r0], u_1, color='tab:green', label=r'$u_{1}(r)$')
    axs[1].plot(r[index_r0:index_r1], u_2, color='tab:red', label=r'$u_{2}(r)$')
    axs[1].plot(r[index_r1:], u_3, color='tab:orange', label=r'$u_{3}(r)$')
    axs[1].plot(r, pot, 'k--', label=r'$V(r)$')

    axs[1].set_xlabel('r (fm)')
    axs[1].set_ylabel(r'V(r)', rotation=0, labelpad=20)
    axs[1].legend()

    axs[1].set_xlim(0, 20)
    axs[1].set_ylim(-120, 60)

    # Remove the upper and right spines
    axs[1].spines['top'].set_visible(False)
    axs[1].spines['right'].set_visible(False)

    plt.tight_layout()

    plt.savefig('temp/barrier_energy_{}.png'.format(idx))

    plt.close(fig)

In [7]:
# Create a gif
images = natsorted( glob('temp/barrier_energy*.png') )
gif = 'gifs/barrier_energy.gif'

with imageio.get_writer(gif, mode='I') as writer:
    for filename in images:
        image = imageio.imread(filename)
        writer.append_data(image)

/var/folders/13/_3g2lmwx6xl3zshjv1d7yz4c0000gn/T/ipykernel_26939/957114650.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [8]:
for idx in range( nframes ):

    v0_min = 10
    v0_max = 100
    V0 = v0_min + idx * (v0_max - v0_min) / nframes

    # Select the potential and the radius
    V1 = 10
    R0, R1 = 3, 8
    e0 = 5

    # Plot the solution for different values of E, Vo and a
    energies = np.linspace(1, 800, 400)
    shift = np.zeros( energies.shape )
    transmission = np.zeros( energies.shape )

    Emin, Emax, n = 2, 10, 500
    energies, shift, transmission = np.zeros(n), np.zeros(n), np.zeros(n)
    for j in range( n ):
        e = Emin + j * (Emax - Emin) / n
        _, ps, _, T = compute(e, 0)
        energies[j], shift[j], transmission[j] =  e, ps[0], T

    # Check in y array if the value jumps by more than 1, if so remove it
    for i in range( 1, len(shift) ):
        if( abs(shift[i] - shift[i-1]) > 1 ):
            shift[i:] = [x-(shift[i] - shift[i-1]) for x in shift[i:]]

    uL, _, _, _ = compute(e0, 0)
    pot = [V(x) for x in r]

    # Find the closest index to e0
    index = np.abs(energies - e0).argmin()
    ps, T = shift[index], transmission[index]

    # Divide the wavefunction in u1, u2, u3 basing on r0 and r1
    index_r0 = np.argmin( np.abs(r - R0) )
    index_r1 = np.argmin( np.abs(r - R1) )

    u_1 = uL[0][:index_r0]
    u_2 = uL[0][index_r0:index_r1]
    u_3 = uL[0][index_r1:]

    # Create two figures
    fig, axs = plt.subplots( 2, 1, figsize=(7, 9) )
    ax1, ax2 = axs[0], axs[0].twinx( )

    # Plot the transmission probability
    ax1.plot(energies, transmission, color='tab:blue')
    ax1.scatter(e0, T, color='tab:blue')
    ax1.set_yscale( "log" )
    ax1.set_xlabel('Center-of-Mass Energy (MeV)')
    ax1.set_ylabel(r'$|\frac{A}{C}| ^{2}$', color='tab:blue', rotation=0, labelpad=20, fontsize=15)

    # Plot the phase shift
    ax2.plot(energies, shift, color='tab:red')
    ax2.scatter(e0, ps, color='tab:red')
    ax2.set_ylabel(r'$\delta _0$', color='tab:red', rotation=0, labelpad=20)

    ax1.set_title(r'E$_0$ = 5 MeV, r$_0$ = 3 fm, V$_1$ = 10 MeV, r$_1$ = 8 fm')
    ax1.set_xlim(2, 10)

    axs[1].plot(r[:index_r0], u_1, color='tab:green', label=r'$u_{1}(r)$')
    axs[1].plot(r[index_r0:index_r1], u_2, color='tab:red', label=r'$u_{2}(r)$')
    axs[1].plot(r[index_r1:], u_3, color='tab:orange', label=r'$u_{3}(r)$')
    axs[1].plot(r, pot, 'k--', label=r'$V(r)$')

    axs[1].set_xlabel('r (fm)')
    axs[1].set_ylabel(r'V(r)', rotation=0, labelpad=20)
    axs[1].legend()

    axs[1].set_xlim(0, 20)
    axs[1].set_ylim(-120, 60)

    # Remove the upper and right spines
    axs[1].spines['top'].set_visible(False)
    axs[1].spines['right'].set_visible(False)

    plt.tight_layout()

    plt.savefig('temp/barrier_v0_{}.png'.format(idx))

    plt.close(fig)

In [9]:
# Create a gif
images = natsorted( glob('temp/barrier_v0*.png') )
gif = 'gifs/barrier_v0.gif'

with imageio.get_writer(gif, mode='I') as writer:
    for filename in images:
        image = imageio.imread(filename)
        writer.append_data(image)

/var/folders/13/_3g2lmwx6xl3zshjv1d7yz4c0000gn/T/ipykernel_26939/2344691842.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [10]:
for idx in range( nframes ):

    v1_min = 0
    v1_max = 20
    V1 = v1_min + idx * (v1_max - v1_min) / nframes

    # Select the potential and the radius
    V0 = 50
    R0, R1 = 3, 8
    e0 = 5

    # Plot the solution for different values of E, Vo and a
    energies = np.linspace(1, 800, 400)
    shift = np.zeros( energies.shape )
    transmission = np.zeros( energies.shape )

    Emin, Emax, n = 2, 10, 500
    energies, shift, transmission = np.zeros(n), np.zeros(n), np.zeros(n)
    for j in range( n ):
        e = Emin + j * (Emax - Emin) / n
        _, ps, _, T = compute(e, 0)
        energies[j], shift[j], transmission[j] =  e, ps[0], T

    # Check in y array if the value jumps by more than 1, if so remove it
    for i in range( 1, len(shift) ):
        if( abs(shift[i] - shift[i-1]) > 1 ):
            shift[i:] = [x-(shift[i] - shift[i-1]) for x in shift[i:]]

    uL, _, _, _ = compute(e0, 0)
    pot = [V(x) for x in r]

    # Find the closest index to e0
    index = np.abs(energies - e0).argmin()
    ps, T = shift[index], transmission[index]

    # Divide the wavefunction in u1, u2, u3 basing on r0 and r1
    index_r0 = np.argmin( np.abs(r - R0) )
    index_r1 = np.argmin( np.abs(r - R1) )

    u_1 = uL[0][:index_r0]
    u_2 = uL[0][index_r0:index_r1]
    u_3 = uL[0][index_r1:]

    # Create two figures
    fig, axs = plt.subplots( 2, 1, figsize=(7, 9) )
    ax1, ax2 = axs[0], axs[0].twinx( )

    # Plot the transmission probability
    ax1.plot(energies, transmission, color='tab:blue')
    ax1.scatter(e0, T, color='tab:blue')
    ax1.set_yscale( "log" )
    ax1.set_xlabel('Center-of-Mass Energy (MeV)')
    ax1.set_ylabel(r'$|\frac{A}{C}| ^{2}$', color='tab:blue', rotation=0, labelpad=20, fontsize=15)

    # Plot the phase shift
    ax2.plot(energies, shift, color='tab:red')
    ax2.scatter(e0, ps, color='tab:red')
    ax2.set_ylabel(r'$\delta _0$', color='tab:red', rotation=0, labelpad=20)

    ax1.set_title(r'E$_0$ = 5 MeV, V$_0$ = 50 MeV, r$_0$ = 3 fm, r$_1$ = 8 fm')
    ax1.set_xlim(2, 10)

    axs[1].plot(r[:index_r0], u_1, color='tab:green', label=r'$u_{1}(r)$')
    axs[1].plot(r[index_r0:index_r1], u_2, color='tab:red', label=r'$u_{2}(r)$')
    axs[1].plot(r[index_r1:], u_3, color='tab:orange', label=r'$u_{3}(r)$')
    axs[1].plot(r, pot, 'k--', label=r'$V(r)$')

    axs[1].set_xlabel('r (fm)')
    axs[1].set_ylabel(r'V(r)', rotation=0, labelpad=20)
    axs[1].legend()

    axs[1].set_xlim(0, 20)
    axs[1].set_ylim(-120, 60)

    # Remove the upper and right spines
    axs[1].spines['top'].set_visible(False)
    axs[1].spines['right'].set_visible(False)

    plt.tight_layout()

    plt.savefig('temp/barrier_v1_{}.png'.format(idx))

    plt.close(fig)

In [11]:
# Create a gif
images = natsorted( glob('temp/barrier_v1*.png') )
gif = 'gifs/barrier_v1.gif'

with imageio.get_writer(gif, mode='I') as writer:
    for filename in images:
        image = imageio.imread(filename)
        writer.append_data(image)

/var/folders/13/_3g2lmwx6xl3zshjv1d7yz4c0000gn/T/ipykernel_26939/3540686074.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [12]:
for idx in range( nframes ):

    r0_min = 1
    r0_max = 5
    R0 = r0_min + idx * (r0_max - r0_min) / nframes

    # Select the potential and the radius
    V0, V1 = 50, 10
    R1 = 8
    e0 = 5

    # Plot the solution for different values of E, Vo and a
    energies = np.linspace(1, 800, 400)
    shift = np.zeros( energies.shape )
    transmission = np.zeros( energies.shape )

    Emin, Emax, n = 2, 10, 500
    energies, shift, transmission = np.zeros(n), np.zeros(n), np.zeros(n)
    for j in range( n ):
        e = Emin + j * (Emax - Emin) / n
        _, ps, _, T = compute(e, 0)
        energies[j], shift[j], transmission[j] =  e, ps[0], T

    # Check in y array if the value jumps by more than 1, if so remove it
    for i in range( 1, len(shift) ):
        if( abs(shift[i] - shift[i-1]) > 1 ):
            shift[i:] = [x-(shift[i] - shift[i-1]) for x in shift[i:]]

    uL, _, _, _ = compute(e0, 0)
    pot = [V(x) for x in r]

    # Find the closest index to e0
    index = np.abs(energies - e0).argmin()
    ps, T = shift[index], transmission[index]

    # Divide the wavefunction in u1, u2, u3 basing on r0 and r1
    index_r0 = np.argmin( np.abs(r - R0) )
    index_r1 = np.argmin( np.abs(r - R1) )

    u_1 = uL[0][:index_r0]
    u_2 = uL[0][index_r0:index_r1]
    u_3 = uL[0][index_r1:]

    # Create two figures
    fig, axs = plt.subplots( 2, 1, figsize=(7, 9) )
    ax1, ax2 = axs[0], axs[0].twinx( )

    # Plot the transmission probability
    ax1.plot(energies, transmission, color='tab:blue')
    ax1.scatter(e0, T, color='tab:blue')
    ax1.set_yscale( "log" )
    ax1.set_xlabel('Center-of-Mass Energy (MeV)')
    ax1.set_ylabel(r'$|\frac{A}{C}| ^{2}$', color='tab:blue', rotation=0, labelpad=20, fontsize=15)

    # Plot the phase shift
    ax2.plot(energies, shift, color='tab:red')
    ax2.scatter(e0, ps, color='tab:red')
    ax2.set_ylabel(r'$\delta _0$', color='tab:red', rotation=0, labelpad=20)

    ax1.set_title(r'E$_0$ = 5 MeV, V$_0$ = 50 MeV, V$_1$ = 10 MeV, r$_1$ = 8 fm')
    ax1.set_xlim(2, 10)

    axs[1].plot(r[:index_r0], u_1, color='tab:green', label=r'$u_{1}(r)$')
    axs[1].plot(r[index_r0:index_r1], u_2, color='tab:red', label=r'$u_{2}(r)$')
    axs[1].plot(r[index_r1:], u_3, color='tab:orange', label=r'$u_{3}(r)$')
    axs[1].plot(r, pot, 'k--', label=r'$V(r)$')

    axs[1].set_xlabel('r (fm)')
    axs[1].set_ylabel(r'V(r)', rotation=0, labelpad=20)
    axs[1].legend()

    axs[1].set_xlim(0, 20)
    axs[1].set_ylim(-120, 60)

    # Remove the upper and right spines
    axs[1].spines['top'].set_visible(False)
    axs[1].spines['right'].set_visible(False)

    plt.tight_layout()

    plt.savefig('temp/barrier_r0_{}.png'.format(idx))

    plt.close(fig)

In [13]:
# Create a gif
images = natsorted( glob('temp/barrier_r0*.png') )
gif = 'gifs/barrier_r0.gif'

with imageio.get_writer(gif, mode='I') as writer:
    for filename in images:
        image = imageio.imread(filename)
        writer.append_data(image)

/var/folders/13/_3g2lmwx6xl3zshjv1d7yz4c0000gn/T/ipykernel_26939/3750775910.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [14]:
for idx in range( nframes ):

    r1_min = 0
    r1_max = 10
    R1 = r1_min + idx * (r1_max - r1_min) / nframes

    # Select the potential and the radius
    V0, V1 = 50, 10
    R0 = 3
    e0 = 5

    # Plot the solution for different values of E, Vo and a
    energies = np.linspace(1, 800, 400)
    shift = np.zeros( energies.shape )
    transmission = np.zeros( energies.shape )

    Emin, Emax, n = 2, 10, 500
    energies, shift, transmission = np.zeros(n), np.zeros(n), np.zeros(n)
    for j in range( n ):
        e = Emin + j * (Emax - Emin) / n
        _, ps, _, T = compute(e, 0)
        energies[j], shift[j], transmission[j] =  e, ps[0], T

    # Check in y array if the value jumps by more than 1, if so remove it
    for i in range( 1, len(shift) ):
        if( abs(shift[i] - shift[i-1]) > 1 ):
            shift[i:] = [x-(shift[i] - shift[i-1]) for x in shift[i:]]

    uL, _, _, _ = compute(e0, 0)
    pot = [V(x) for x in r]

    # Find the closest index to e0
    index = np.abs(energies - e0).argmin()
    ps, T = shift[index], transmission[index]

    # Divide the wavefunction in u1, u2, u3 basing on r0 and r1
    index_r0 = np.argmin( np.abs(r - R0) )
    index_r1 = np.argmin( np.abs(r - R1) )

    u_1 = uL[0][:index_r0]
    u_2 = uL[0][index_r0:index_r1]
    u_3 = uL[0][index_r1:]

    # Create two figures
    fig, axs = plt.subplots( 2, 1, figsize=(7, 9) )
    ax1, ax2 = axs[0], axs[0].twinx( )

    # Plot the transmission probability
    ax1.plot(energies, transmission, color='tab:blue')
    ax1.scatter(e0, T, color='tab:blue')
    ax1.set_yscale( "log" )
    ax1.set_xlabel('Center-of-Mass Energy (MeV)')
    ax1.set_ylabel(r'$|\frac{A}{C}| ^{2}$', color='tab:blue', rotation=0, labelpad=20, fontsize=15)

    # Plot the phase shift
    ax2.plot(energies, shift, color='tab:red')
    ax2.scatter(e0, ps, color='tab:red')
    ax2.set_ylabel(r'$\delta _0$', color='tab:red', rotation=0, labelpad=20)

    ax1.set_title(r'E$_0$ = 5 MeV, V$_0$ = 50 MeV, r$_0$ = 8 fm, V$_1$ = 10 MeV')
    ax1.set_xlim(2, 10)

    axs[1].plot(r[:index_r0], u_1, color='tab:green', label=r'$u_{1}(r)$')
    axs[1].plot(r[index_r0:index_r1], u_2, color='tab:red', label=r'$u_{2}(r)$')
    axs[1].plot(r[index_r1:], u_3, color='tab:orange', label=r'$u_{3}(r)$')
    axs[1].plot(r, pot, 'k--', label=r'$V(r)$')

    axs[1].set_xlabel('r (fm)')
    axs[1].set_ylabel(r'V(r)', rotation=0, labelpad=20)
    axs[1].legend()

    axs[1].set_xlim(0, 20)
    axs[1].set_ylim(-120, 60)

    # Remove the upper and right spines
    axs[1].spines['top'].set_visible(False)
    axs[1].spines['right'].set_visible(False)

    plt.tight_layout()

    plt.savefig('temp/barrier_r1_{}.png'.format(idx))

    plt.close(fig)

In [15]:
# Create a gif
images = natsorted( glob('temp/barrier_r1*.png') )
gif = 'gifs/barrier_r1.gif'

with imageio.get_writer(gif, mode='I') as writer:
    for filename in images:
        image = imageio.imread(filename)
        writer.append_data(image)

/var/folders/13/_3g2lmwx6xl3zshjv1d7yz4c0000gn/T/ipykernel_26939/3988569363.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
